In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy import stats

# 1. Carregar os datasets
df_netflix = pd.read_csv('netflix_titles.csv')
df_imdb = pd.read_csv('imdb.csv')
df_rottentomatoes = pd.read_csv('rot_tmt.csv')

# Exibir as colunas disponíveis para verificação
print("Netflix columns:", df_netflix.columns)
print("IMDB columns:", df_imdb.columns)
print("Rotten Tomatoes columns:", df_rottentomatoes.columns)

# 2. Renomear colunas para facilitar a junção
df_imdb.rename(columns={'Series_Title': 'title', 'Released_Year': 'release_year'}, inplace=True)
df_rottentomatoes.rename(columns={'movie_title': 'title'}, inplace=True)

# Converter 'original_release_date' para apenas o ano em Rotten Tomatoes
df_rottentomatoes['release_year'] = pd.to_datetime(df_rottentomatoes['original_release_date'], errors='coerce').dt.year

# Garantir que 'release_year' esteja no formato correto em todos os datasets
df_netflix['release_year'] = pd.to_numeric(df_netflix['release_year'], errors='coerce')
df_imdb['release_year'] = pd.to_numeric(df_imdb['release_year'], errors='coerce')
df_rottentomatoes['release_year'] = pd.to_numeric(df_rottentomatoes['release_year'], errors='coerce')

# Verificar se todas as colunas 'release_year' foram convertidas corretamente
print(df_netflix['release_year'].dtype)
print(df_imdb['release_year'].dtype)
print(df_rottentomatoes['release_year'].dtype)

# Combinar datasets novamente
combined_data = pd.merge(df_netflix, df_imdb, on=['title', 'release_year'], how='inner')
combined_data = pd.merge(combined_data, df_rottentomatoes, on=['title', 'release_year'], how='inner')

# Exibir os primeiros dados combinados para verificação
print("Dados combinados:")
print(combined_data.head())

# Exibir as colunas renomeadas e verificadas
print("IMDB columns after rename:", df_imdb.columns)
print("Rotten Tomatoes columns after rename and date conversion:", df_rottentomatoes.columns)

# 3. Combinar datasets com base no título e ano de lançamento
combined_data = pd.merge(df_netflix, df_imdb, on=['title', 'release_year'], how='inner')
combined_data = pd.merge(combined_data, df_rottentomatoes, on=['title', 'release_year'], how='inner')

# Exibir os primeiros dados combinados para verificação
print("Dados combinados:")
print(combined_data.head())

# 4. Separar elencos e remover espaços
combined_data['cast_netflix'] = combined_data['cast_netflix'].str.split(',').apply(lambda x: [name.strip() for name in x])
combined_data['cast_imdb'] = combined_data['cast_imdb'].str.split(',').apply(lambda x: [name.strip() for name in x])
combined_data['cast_rottentomatoes'] = combined_data['actors'].str.split(',').apply(lambda x: [name.strip() for name in x])

# 5. Identificação de atores em comum
def common_actors(row):
    return list(set(row['cast_netflix']).intersection(row['cast_imdb'], row['cast_rottentomatoes']))

combined_data['common_actors'] = combined_data.apply(common_actors, axis=1)

# 6. Análise Exploratória de Dados (EDA)
# Calcular o número de atores em comum por filme
combined_data['num_common_actors'] = combined_data['common_actors'].apply(len)

# Visualização: Distribuição de atores em comum
sns.histplot(combined_data['num_common_actors'], bins=10)
plt.title("Distribuição de Atores em Comum")
plt.xlabel("Número de Atores em Comum")
plt.ylabel("Frequência")
plt.show()

# 7. Aplicação de Regressão Linear
# Variáveis: Número de atores em comum e a avaliação no IMDb
X = combined_data[['num_common_actors']]
y = combined_data['IMDB_Rating']  # Supondo que esta coluna exista no dataset

# Modelo de Regressão Linear
reg = LinearRegression().fit(X, y)
print(f"Coeficiente: {reg.coef_}, Intercepto: {reg.intercept_}")

# 8. Teste de Hipóteses
# Separar filmes com muitos e poucos atores em comum
high_common_actors = combined_data[combined_data['num_common_actors'] > 3]
low_common_actors = combined_data[combined_data['num_common_actors'] <= 3]

# Teste T para comparar as médias de avaliações entre os dois grupos
t_stat, p_value = stats.ttest_ind(high_common_actors['IMDB_Rating'], low_common_actors['IMDB_Rating'])
print(f"Teste T: {t_stat}, Valor p: {p_value}")

Netflix columns: Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')
IMDB columns: Index(['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')
Rotten Tomatoes columns: Index(['rotten_tomatoes_link', 'movie_title', 'movie_info',
       'critics_consensus', 'content_rating', 'genres', 'directors', 'authors',
       'actors', 'original_release_date', 'streaming_release_date', 'runtime',
       'production_company', 'tomatometer_status', 'tomatometer_rating',
       'tomatometer_count', 'audience_status', 'audience_rating',
       'audience_count', 'tomatometer_top_critics_count',
       'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count'],
      dtype='object')
int64
float

KeyError: 'cast_netflix'